# **Using TensorFlow to finetune a GPT-neo model with United Nations General Assembly resolutions**

*Authors*:
Felix Gröner, Nils Axe

# Summary
The goal of this project is to finetune a generative pretrained transformer (GPT) and generate United Nations (UN) resolutions.

###Data
The United Nations started publishing some of their documents in a machine-readable format. More specifically, we retrieved resolutions adopted by the General Assembly in their 75th session (2020/2021). The XML files were downloaded from the official github repository.

UN Resolution have a very specific structure:
 

*   They start with the adopting body, which in our case is always "The GeneralAssembly,"
*   Then, the first half contains the preambular clauses. They usually start with a verb in present progressive (-ing) and end with a comma.
*   The second halfpart contains the operative clauses, which are enumerated. They each start with a verb in simple present and end with a semicolon. The final clause ends with a full stop.




Our XML parser reads the xml and extracts only the relevant parts, i.e. the clauses. Then it concatenates the clauses of all resolutions to a single .txt-file, which we use as training data.

###Model
We use GPT-neo and finetune it to adopt the style of the UN resolutions. More specifically, the pretrained model is GPT3_XL by EleutherAI with 1,3 billion parameters.
Our training takes place in the Google Cloud

### Prompts
To generate a random resolution, the prompt should be "The General Assembly,".

We were able to generate resolutions for a specific topic by setting the prompt to the title of that topic and then adding "The General Assembly,". These work best if the topics are similar to those actually discussed at the United Nations.



###Results
The results have been analyzed by an expert familiar with UN resolutions and evaluated in three aspects: format, content structure and content coherence. A sample can be found at the bottom of this notebook.


The generated resolutions are in accordance with the aforementioned specific structure of UN resolutions. Viable verbs in the correct tenses are used and usually the correct punctuation is set. There sometimes are minor hiccups where a footnote interferes with a clause.


The content structure is surprisingly accurate. The first preambular clauses usually refers to past resolutions while the last operative clause refers to the provisional future agenda.


The content coherence is not as good and gives away that it is a generated resolution. However, this mostly shows in details, which are discovered only by experts. Resolutions codes, adoption dates, resolution titles and resolution content don't always match. Also, memorial days and their dates sometimes don't match. The content appears to be held so general that it can't really be incorrect (like horoscopes) and sometimes includes elements not commonly found in UN documents.

#Dataset preparation

In [ ]:
# Download resolutions from github

!git clone https://github.com/UNxml/GAresolutions.git

##XML Parser

In [ ]:
import re
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup

# removes special characters from a string
def clean_clause (text) :
    text = re.sub(r'<[^<]+>', '', text)
    text = text.replace('\\n', '')
    text = text.replace('b\'', '')
    text = text.replace('\\t', '')
    text = text.replace('\'', '')
    #text = text.strip()
    text = " ".join(text.split())
    return text

# appends clauses from res_path to a .txt file at target_path
def append_res(res_path, target_path) :
  clauses = ["\n"]

  tree = ET.parse(res_path)
  root = tree.getroot()
  ns = {"ns":"http://docs.oasis-open.org/legaldocml/ns/akn/3.0"}

  statement = root.find('ns:statement', ns)

  # Resolution Title
  preface = statement.find("ns:preface", ns)
  paragraph = preface.find("ns:p", ns)
  doctitle = paragraph.find("ns:docTitle", ns)
  clauses.append(clean_clause(str(ET.tostring(doctitle))))

  clauses.append("The General Assembly,")

  # Preambular Clauses
  preamble = statement.find("ns:preamble", ns)
  for container in preamble.findall('ns:container', ns) :
    clause = container.find('ns:p', ns)
    #test = clause.find('ns:span', ns)
    text = str(ET.tostring(clause))
    #print(.replace('<[^<]+>', ''))
    #print()

    clauses.append(clean_clause(text))

  # Operative Clauses
  main_body = statement.find("ns:mainBody", ns)
  for paragraph in main_body.findall('ns:paragraph', ns) :
    text = str(ET.tostring(paragraph))

    clauses.append(clean_clause(text))

  # write to txt file
  with open(target_path, 'a') as txt_file: # change this to 'a' for appending all res to the same txt
    for clause in clauses :
      txt_file.write("\n"+ BeautifulSoup(clause).text)

## Extracting and Concatenating Resolutions and Clauses

In [ ]:
import os
# now go through all resolution xml files and append their clauses to the txt file

target_file = "concat_resolutions.txt"
#res_folder_path = 'xml/English/'
res_folder_path = 'GAresolutions/75session/English/'

filenames = os.listdir(res_folder_path)

# HINT : This extracts a single resolution. The code below that extracts all resolutions of one session
#append_res(res_folder_path+filenames[0], target_file)

for filename in filenames :
  try :
    append_res(res_folder_path+filename, target_file)
  except:
    print("Problem with file", filename)

print("Concatenated", len(filenames), "resolutions")

#Finetuning GPT-neo

##Downloading GPT-neo

In [ ]:
#We are using the EleutherAI GPT-neo implementation

%tensorflow_version 2.x
!git clone https://github.com/EleutherAI/GPTNeo
%cd GPTNeo
!pip3 install -q -r requirements.txt
pretrained_model = None
dataset = None

## Setting up Google Cloud
We are using Google Cloud Buckets as permanent storage of the model data.

In [ ]:
#Authentication
from google.colab import auth
auth.authenticate_user()
!gcloud init

In [ ]:
#Setting path to bucket
path_to_cloud_bucket = 'gs://BUCKETPATH/'

##Setting up the dataset

In [ ]:
  dataset = 'custom'
  dataset_path = '/content/' #Path to the folder containing the data
  dataset_name = 'concat_resoulutions.txt' #Name of the dataset
  out_name = dataset_name + "_tokenized"

### Tokenizing and uploading the dataset

In [ ]:
# Tokenize Data with GPT2
#!python data/create_tfrecords.py --input_dir /content/GPTNeo/$dataset_path --name $dataset_name --files_per 1000 --output_dir $out_name --write_dataset_config --processes 1
!python data/create_tfrecords.py --input_dir /content/ --name $dataset_name --files_per 1000 --output_dir $out_name --write_dataset_config --processes 1

# copy the data to the bucket
if not path_to_cloud_bucket.endswith('/'):
  path_to_cloud_bucket += '/'
copy_loc = path_to_cloud_bucket + "datasets/" + dataset
!gsutil -m cp -r /content/GPTNeo/$out_name $copy_loc
!gsutil ls $path_to_cloud_bucket

In [ ]:
#Setting dataset-configs

%%writefile configs/dataset_configs/concat_resolutions.json

{
  "path": "gs://BUCKETPATH/datasets/custom/concat_resoulutions.txt*.tfrecords",
  "eval_path": "",
  "n_vocab": 50256,
  "tokenizer_is_pretrained": true,
  "tokenizer_path": "gpt2",
  "eos_id": 50256,
  "padding_id": 50257
}


## Setting up the model

In [ ]:
#Setting up the model-configs

%%writefile configs/GPT3_XL.json

{
    "n_head": 16,
    "n_vocab": 50257,
    "embed_dropout": 0,
    "lr": 0.0002,
    "lr_decay": "cosine",
    "warmup_steps": 3000,
    "beta1": 0.9,
    "beta2": 0.95,
    "epsilon": 1e-8,
    "opt_name": "adam",
    "weight_decay": 0,
    "train_batch_size": 256,
    "attn_dropout": 0,
    "train_steps": 600000,
    "eval_steps": 0,
    "predict_steps": 1,
    "res_dropout": 0,
    "eval_batch_size": 4,
    "predict_batch_size": 1,
    "iterations": 100,
    "n_embd": 2048,
    "datasets": [["pile", null, null, null]],
    "model": "GPT",
    "model_path": "gs://BUCKETPATH/GPT3_XL/",
    "n_ctx": 2048,
    "n_layer": 24,
    "scale_by_depth": true,
    "scale_by_in": false,
    "attention_types" :  [[["global", "local"],12]],
    "mesh_shape": "x:4,y:2",
    "layout": "intermediate_expanded:x,heads:x,vocab:n_vocab,memory_length:y,embd:y",
    "activation_function": "gelu",
    "recompute_grad": true,
    "gradient_clipping": 1.0,
    "tokens_per_mb_per_replica": 2048,
    "precision": "bfloat16"
}

In [ ]:
#Downloading the pretrained weights
!wget -m -np -c -U "eye02" -w 2 -R "index.html*" "https://the-eye.eu/public/AI/gptneo-release/GPT3_XL/"
path_to_local_weights = f"/content/GPTNeo/the-eye.eu/public/AI/gptneo-release/GPT3_XL"

In [ ]:
#Uploading the weights to the bucket
bucket_base = "gs://" + path_to_cloud_bucket.replace('gs://', '').split('/')[0]
!gsutil -m cp -r $path_to_local_weights $bucket_base

In [ ]:
#Listing bucket contents
!gsutil ls $bucket_base

###Colab specific configs

In [ ]:
import json
from pprint import pprint

path_to_model = "gs://BUCKETPATH/GPT3_XL/"
batch_size = 8 #lower uses less memory (8)
dset = "concat_resolutions" #name of the dataset-config file
mesh_shape = "x:4,y:2" #DONT TOUCH; specifies the way the model will be divided up across the TPU cores (x:4,y:2)
train_steps = 1000 #how many training steps are done (1000)
steps_per_checkpoint = 500 #steps per checkpoint saved
start_step = 362000 #362000

if path_to_model == "":
  path_to_model = f'{bucket_base.strip("/")}/{pretrained_model}'
print(f'MODEL PATH: {path_to_model}\n')

if dset == "" and dataset != "Sampling_Only":
  dset = dataset
elif dataset is None and dset == "":
  dset = "pile"

def pad_to_multiple_of(n, mult):
  """
  pads n to a multiple of mult
  """
  extra = n % mult
  if extra > 0:
      n = n + mult - extra
  return n

with open(f'{path_to_local_weights}/config.json', 'r') as f:
  data = json.load(f)
  pprint(data)
  dset_val = [[dset, None, None, None]] if dset != "" else data["datasets"]
  mods = {
          "mesh_shape": mesh_shape,
          "layout": "intermediate_expanded:x,heads:x,memory_length:y,embd:y",
          "model_path": path_to_model,
          "datasets": dset_val,
          "train_steps": start_step + train_steps,
          "eval_steps": 0,
          "train_batch_size": batch_size,
          "predict_batch_size": batch_size
        }
  data.update(mods)
  print('\n--->\n')
  pprint(data)
  with open(f'configs/{pretrained_model}.json', 'w') as outfile:
    json.dump(data, outfile, indent=2)

#Finetuning the model

In [ ]:
!python3 main.py --model $pretrained_model --steps_per_checkpoint $steps_per_checkpoint --tpu colab

#Predicting

In [ ]:
#To pass in a prompt, save it to a .txt file, and pass in the name of the file with the --prompt flag.
!python3 main.py --model $pretrained_model --steps_per_checkpoint 500 --tpu colab --predict --prompt /content/prompt.txt

#Annex I: Sample Generated Resolution

The General Assembly,

Recalling its resolutions 62/144 of 23 December 2007, 63/144 of 22 December 2008, 64/159 of 20 December 2009, 65/195 of 19 December 2010, 66/155 of 19 December 2011, 67/143 of 20 December 2012, 68/153 of 19 December 2013, 69/152 of 19 December 2014, 70/164 of 21 December 2015, 71/164 of 21 December 2016, 72/155 of 20 December 2017, 73/154 of 20 December 2018 and 74/153 of 19 December 2019 on the role of the United Nations in promoting a culture of peace and disarmament within the international community,

Reaffirming its resolution 70/1 of 25 September 2015, entitled "Transforming our world: the 2030 Agenda for Sustainable Development", in which it adopted a comprehensive, far-reaching and people-centred set of universal and transformative Sustainable Development Goals and targets, its commitment to working tirelessly for the full implementation of the Agenda by 2030, its recognition that eradicating poverty in all its forms and dimensions, including extreme poverty, is the greatest global challenge and an indispensable requirement for sustainable development, its commitment to achieving sustainable development in its three dimensions – economic, social and environmental – in a balanced and integrated manner, and to building upon the achievements of the Millennium Development Goals and seeking to address their unfinished business,

Reaffirming also its resolution 69/313 of 27 July 2015 on the Addis Ababa Action Agenda of the Third International Conference on Financing for Development, which is an integral part of the 2030 Agenda for Sustainable Development, supports and complements it, helps to contextualize its means of implementation targets with concrete policies and actions, and reaffirms the strong political commitment to address the challenge of financing and creating an enabling environment at all levels for sustainable development in the spirit of global partnership and solidarity,

Recognizing the important role of regional and subregional organizations and networks in promoting a culture of peace and disarmament,Abbreviating°A/HRC/17/36 layover on the way to the Summit, the status of the 8,300- point mixedmaps on the way to the Summit, the underlining message of the video message from the Secretary-General on the occasion of the 75th anniversary of the International Committee of the Fourth World to Combat Nuclear Weapons, to prevent an armsECHA/SGB/40/Rev.1.pdf, see E/CN.4/2016/3.

Noting with great concern the threat to human health, safety and well-being caused by the coronavirus disease (COVID-19) pandemic, as well as the severe disruption to societies and economies and the devastating impact on lives and livelihoods, and that the poorest and most vulnerable are the hardest hit by the pandemic, reaffirming the ambition to get back on track to achieve the Sustainable Development Goals by designing sustainable and inclusive recovery strategies to accelerate progress towards the full implementation of the 2030 Agenda and to help to reduce the risk of future shocks, and recognizing that the COVID-19 pandemic requires a global response based on unity, solidarity and renewed multilateral cooperation,

Recognizing the indispensable role of information and communications technology (including the Internet) in promoting a culture of peace and disarmament, increasing transparency and knowledge-sharing, and contributing to better and more inclusive decision-making,

Encouraging a more inclusive and understanding approach to disarmament, and de-emphasizing doctrines, schoolings and philosophies that promote doctrines, schoolings and philosophies that promote for example unilateralism, unilateralism in war, cold-war doctrines, sequentialism, Shangri-La doctrines and martial principles,

Recalling the Declaration of Principles on Interim Self‐Government Arrangements on international peace and disarmament of the League of Arab States and the Organization for Security and Cooperation, and welcoming the ongoing efforts of the League and the Organization to advance the culture of peace and disarmament within the international system,

Recalling also the 2030 Agenda for Sustainable Development,Resolution 70/1. and noting that its means of implementation targets are not fully achieved at the individual level and are linked to collective outcomes,

Encouraging the systematic and multilateral promotion of a culture of peace and disarmament and their international cooperation, including through peace and security education at all levels,

Recalling that the General Assembly, in its resolution 74/153 of 19 December 2019, requested the Secretary-General to report to the General Assembly at its 80th session on the implementation of the present resolution, and on the role of the United Nations in promoting a culture of peace and disarmament within the international community,

1. Notes the importance of the culture of peace and disarmament in ensuring international peace and security;

2. Recalls the adoption on 21 November 1967 of the United Nations Charter, and encourages the continued promotion and strengthening of the culture of peace and disarmament within the international framework;

3. Decides to follow up on the issues referred to in the present resolution and to consider them in the future, given the importance of the culture of peace and disarmament to the security of States and the international community,

4. Invites Member States to continue to promote the culture of peace and disarmament in all areas of their domestic and international policies, as well as at the regional and global levels;

5. Invites the Secretary-General to convene a United Nations high-level event, with the involvement of relevant United Nations agencies, to discuss the culture of peace and disarmament and its international support, with the aim of promoting a more systematic and international approach to peace and disarmament, and aiming to identify practical ways and means for furthering effective anti-threat policies and programmes, in close collaboration with Member States and other actors engaged in peace and security matters;

6. Decides to include in the provisional agenda of its seventy-sixth session, under the item entitled "General and complete disarmament", the sub-item entitled "Culture of peace and disarmament".


#Annex II: Topic-specific Generated Resolution
*Topic: Risks of Artificial Intelligence*

The General Assembly,

Having considered the Report of the Financial and Economic Inquiry into the Digital Economy of the United Nations Decade of Digital Cooperation (2018–2025),Official Records of the General Assembly, Seventy-fifth Session, Supplement No. 7 (A/75/7).

Recalling the Political Declaration of the high-level meeting of the General Assembly to review progress made in the implementation of the digital economy and to support the Office of the High Representative in this regard, held on 12 December 2018,

Aware of the challenges and opportunities that are associated with the increasing demand for assistance in the digital economy,

1. Reaffirms the digital economy as a cross-cutting dimension of the United Nations agenda for sustainable development, which implies a coherent and comprehensive approach which recognizes the interlinkages between digitalization, affordability, accessibility,128 efficiency, productivity and quality of life, and social inclusion, and is aimed at enabling active participation, inclusive and sustained growth and employment opportunities in the digital economy for all;
2. Stresses that the United Nations system, including the World Health Organization, the United Nations Development Programme, the United Nations Environment Programme, the United Nations Conference on Trade and Development, the United Nations Conference on Trade and Development Science and Technology Report 2000: Tecton (ST/2000/1.3.Add.1 tot. I), Society 8 (Supplement No. 50/1), and the Technology Facilitation Mechanism, as well as intergovernmental and non-governmental organizations, have an important role in fostering the development of e-commerce and e-commerce tools, as well as in bridging the digital divides, in accordance with the 2030 Agenda for Sustainable Development;
3. Invites Member States to leverage e-commerce and payments for social and economic development, in particular in developing countries, and notes in this regard the 452,000 jobs supported by the United Nations e-commerce and paymentsña rlist;s blue economy programme; the United Nations Business Council; the International Fund for Agricultural Development; the International Labour Organization; the United Nations Entity for Gender Equality and the Empowerment of Women (UN-Women) and the Treaty d'arriver pour uny advancement de la femme (Tribute pour unir la femme et l'ensemble des femmes à tous les hommes de toutes les cultures et à toutes les sexualelles féminiles) Initiative 2035; the Global Work in/for a Liveable Planet – Agenda for Sustainable DevelopmentResolution 71/256, annex. and by the United Nations system entities engaged in e-commerce, including the World Health Organization, the United Nations Environment Programme, the United Nations Conference on Trade and Development, the United Nations Conference on Trade and Development Science and Technology Report 2000: Tecton (ST/2000/1.3.Add.1 tot. I), Society 8 (Supplement No. 50/1), and the Technology Facilitation Mechanism;
4. Encourages Governments to prioritize the development of policies and programmes that promote the development of e-commerce and e-commerce tools, including payment systems and agencies, that are accessible to women and girls, and in particular to promote their gender equality and the empowerment of their full, equal and effective participation in e-commerce and the digital economy;
5. Encourages the United Nations system, including the World Health Organization, the United Nations Development Programme, the United Nations Environment Programme, the United Nations Conference on Trade and Development, the United Nations Conference on Trade and Development Science and Technology Report 2000: Tecton (ST/2000/1.3.Add.1 tot. I), Society 8 (Supplement No. 50/1), and the Technology Facilitation Mechanism, as well as intergovernmental and non-governmental organizations, to strengthen its capacity to assist in the implementation of the Programme of Action for the Least Developed Countries for the Decade 2011–2020 for accelerating e-commerce and e-commerce payments, easing e-commerce payments burden and building e-commerce infrastructure and reaching e-commerce users, which could cut costs for countries least developed individual countries, for which payments are made days or months a year, for whom e-commerce payments are made a priority and for other reasons are not sustainable, and in this regard to support management of operations, security, procedural safeguards, and access to services;
6. Welcomes the advancement in the field of artificial intelligence, highlights the advancement of technologies and the growing ability to learn from data and build models that can make informed decisions, contribute to a better world and realize its possibilities, and calls upon the United Nations, all relevant United Nations system entities, regional organizations, science- and technology-based organizations, propitution bodies, civil society, academia and the private sector to consider increasing cooperation and engaging more closely in the development, support and implementation of effective policies and programmes to assist Member States in designing, implementing, monitoring and scaling up efforts to prevent and address ill-motivated scouring, as well as match the benefits of digitalization with the needs of individuals and societies;
7. Invites Member States and entities of the United Nations system to strengthen training and capacity-building programmes for civil society, non-governmental organizations, businesses, academic institutions, the private sector and other relevant actors in the digital economy and to improve interface between science- and technology-based organizations and the United Nations system in this regard;
8. Reaffirms the 2030 Agenda for Sustainable Development, Resolution 70/1. and also reaffirms the goal of ensuring inclusive and equitable quality education and lifelong learning for all to advance the Sustainable Development Goals, and in this regard underlines that economic and socialmm inimical to the incentives of free agents and non-discriminatory among capability, capability differentiated and equal opportunities for effective participation and leaderships and cultural and ethnic pluralismin accordance with the Universal Declaration of Human RightsResolution 217 A (III). and the International Covenant on Economic, Social and Cultural Rights;See resolution 2621 (XXI), annex. and paragraphs 18, 19. and 20. of the Convention on the Rights of the Child,United Nations, Treaty Series, vol. 1577, No. 27531. and calls upon business and other providers of services to promote gender equality and the empowerment of all women and girls as agents of change in the digital economy.
9. Emphasizes the need for strengthening the leadership role of the United Nations as a tool for the digital economy and for inclusion in the digital economy, calls upon the entities of the United Nations system to continue to build capacity in the digital economy and to promote the utilization of the latestm imidatingly valetsq and petascale technologies andto gearup to support the effort to includeansk women in the digital economy;
10. Invites Member States and relevant institutions to promote the democratization of the process for artificial intelligence, urges them to ensure gender equality at all levels, to institutionalize the process and to promote its full and equal participation and leadership in different sectors of the digital economy, as well as in the design of policies and programmes and to integrate artificial intelligence and machine-learning into their policies, planning and strategies;
11. Reiterates the need for capacity-building in this regard, and emphasizes the importance of infringement prevention and unitq development;
12. Expresses concern that, while some artificial intelligence (AI) systems can make better decisions than human beings, there is no substitute for human judgment and decision-making, and that there is a need to train more and more systems to operate without any human input or adjustment, and that there is a risk offield-based, data-driven, evidence-based, andmachineskined accuracyq and of making decisions that are not in accordance with the best interests of the individual systems or the collective human beings and that are not inborn, but rather are learned through training and experience;
13. Reaffirms the digital economy mandate of the Office of the High Representative and the importance of promoting the full and equal use of the Internet and e-commerce in accordance with the United Nations Digital Agenda,Resolution 70/175. and of integrating the Internet and e-commerce into all development and human rights policies,Resolution 72/193, annex. and calls upon the entities of the United Nations system to support efforts of Member States to achieve the fullest possible use of the Internet and e-commerce for financial services, improved e-commerce and e-commerce payments and trading, access to information and communications services and the exchange of information, knowledge, science andt echnologyenabled, with a view to supporting inclusive and equitable development and sustaining progress over the long term, and encourages Member States to developwhitepostel listings and other tools that facilitate e-commerce payments and accessibility, as well as data-driven and evidence-based regulatory and policy choices;
14. Reiterates the importance of design thinking and human-centred design in building systems and achieving greater levels of accuracy,speed, flexibility, accessibility and affordability, andof information and communications technologies for AI development and application, stresses thatAccelerated Be build and Algorithms for Budgetary Scrutiny (Accelerated Bridge)System (ACS) is an example of a human-centred, budgeting and scheduling system that used machine learning to improve its accuracy and efficiency, and that has guided the office in making key budgetary decisions of the United Nations in the 10 years since its conception;
15. Recognizes the importance ofdissemination offresherskills, includingstrategies and approaches to tacklefremontism and excessive delays in payments,burden of paper work, andburden on the personnel and facilities necessary to physically visit eachmandated location, amp;reduction of corruption,fear of arrest, arrests and detention, andpreventing and combating bribery, abuse of authority and organized crime in payments processing;
